In [2]:
import urllib.request
import urllib.parse
import json
import pandas as pd
from datetime import datetime
import os
from openpyxl import load_workbook

##################################
# function to unnest json for each month
#her ay için jsonu çıkarma fonksiyonu
def extract_monthly_data(data):
    num_days = len(data)
    # initialize df_month to store return data
    # veriyi döndrürürken depolmak için df_month oluşumu
    df_month = pd.DataFrame()
    for i in range(num_days):
        # extract this day
        #bugünü ayıklamak
        d = data[i]
        # astronomy data is the same for the whole day
        #aynı gün için astronomi bilgileri 
        astr_df = pd.DataFrame(d['astronomy'])
        # hourly data; temperature for each hour of the day
        #saatlik veri ; günün hersaatinin sıcaklığı
        hourly_df = pd.DataFrame(d['hourly'])
        # this wanted_key will be duplicated and use 'ffill' to fill up the NAs
        #bu NA ları doldurmak için 'ffill' kullanacak ve çoğaltacak olan istenen_anahtar(kelime)
        wanted_keys = ['date', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour']  # The keys you want
        subset_d = dict((k, d[k]) for k in wanted_keys if k in d)
        this_df = pd.DataFrame(subset_d, index=[0])
        df = pd.concat([this_df.reset_index(drop=True), astr_df], axis=1)
        # concat selected astonomy columns with hourly data
        #saatlik verileri olan seçili astronomi sutunları
        df = pd.concat([df, hourly_df], axis=1)
        df = df.fillna(method='ffill')
        # make date_time columm to proper format
        #formatı sağlamak için date_time sutununu yapmak
        # fill leading zero for hours to 4 digits (0000-2400 hr)
        #4 haneli saatler için 0 ile doldurmak
        df['time'] = df['time'].apply(lambda x: x.zfill(4))
        # keep only first 2 digit (00-24 hr) 
        #sadece saatin ilk iki hanesini almak
        df['time'] = df['time'].str[:2]
        # convert to pandas datetime
        #pandas datetime a çevrilmesi
        df['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        # keep only interested columns
        #sadece ilgilenilen sutunların tutulması 
        col_to_keep = ['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
                        'sunrise', 'sunset','DewPointC', 'FeelsLikeC', 'WindChillC',
                       'cloudcover', 'humidity', 'precipMM', 'tempC', 'visibility', 'windspeedKmph']
        df = df[col_to_keep]
        df = df.loc[:,~df.columns.duplicated()]
        df_month = pd.concat([df_month, df])
    return (df_month)


##################################
# function to retrive data by date range and location
#yer ve tarih sırasıyla veriyi alan fonksiyon 
# default frequency = 1 hr
#standart frekans =  1 saat 
# each month costs 1 request (free trial 500 requests/key, as of 30-May-2019)
def retrieve_this_location(api_key, location, start_date, end_date, frequency, response_cache_path):
    start_time = datetime.now()

    # create list of first day of month for range between start and end dates non-inclusive (open)
    list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
    # convert to Series and add start_date at beginning
    list_mon_begin = pd.concat([pd.Series(pd.to_datetime(start_date)), pd.Series(list_mon_begin)], ignore_index=True)

    # create list of month end dates for range between start and end dates non-inclusive (open)
    list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')
    # convert to Series and add end_date at end
    list_mon_end = pd.concat([pd.Series(list_mon_end), pd.Series(pd.to_datetime(end_date))], ignore_index=True)
    # count number of months to be retrieved
    total_months = len(list_mon_begin)

    # initialize df_hist to store return data
    df_hist = pd.DataFrame()
    for m in range(total_months):
        start_d = str(list_mon_begin[m])[:10]
        end_d = str(list_mon_end[m])[:10]
        file_path = f'{response_cache_path}/{location}_{start_d}_{end_d}'
        if response_cache_path and os.path.exists(file_path):
            print('Reading cached data for ' + location + ': from ' + start_d + ' to ' + end_d)
            with open(f'{response_cache_path}/{location}_{start_d}_{end_d}', 'r') as f:
                json_data = json.load(f)
        else:
            print('Currently retrieving data for ' + location + ': from ' + start_d + ' to ' + end_d)
            url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=' + api_key + '&q=' + location + '&format=json&date=' + start_d + '&enddate=' + end_d + '&tp=' + str(
                frequency)
            json_page = urllib.request.urlopen(url_page, timeout=5)
            json_data = json.loads(json_page.read().decode())

        if response_cache_path:
            with open(f'{response_cache_path}/{location}_{start_d}_{end_d}', 'w') as f:
                json.dump(json_data, f)
        data = json_data['data']['weather']
        # call function to extract json object
        df_this_month = extract_monthly_data(data)
        df_this_month['location'] = location
        df_hist = pd.concat([df_hist, df_this_month])

        time_elapsed = datetime.now() - start_time
        print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    return (df_hist)


##################################
# main function to retrive the data by location list
def retrieve_hist_data(api_key, location, start_date, end_date, frequency, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None):
    result_list = []
    
    print('\n\nRetrieving weather data for ' + location + '\n\n')
    df_this_city = retrieve_this_location(api_key, location, start_date, end_date, frequency, response_cache_path)

    if (location_label == True):
            # add city name as prefix to the colnames
        df_this_city = df_this_city.add_prefix(location + '_')
        df_this_city.columns.values[0] = 'date_time'

    if (export_csv == True):
        df_this_city.to_csv('./' + location + '.csv', header=True, index=False)
        print('\n\nexport ' + location + ' completed!\n\n')

    if (store_df == True):
            # save result as object in the work space
        result_list.append(df_this_city)

    return (result_list)
lokasyonlar = ['usak']
iller = ['erzurum','diyarbakır','adana','gaziantep','samsun','trabzon','manisa','ankara','konya','izmir','antalya','kayseri'
        'bursa','tekirdag','denizli','istanbul']
def hava_cek(locationn):
    frequency=2
    start_date = '01-JAN-2018'
    end_date = '18-MAR-2021'
    api_key = 'ea2f7452d098419f934121404210803'
    hist_weather_data = retrieve_hist_data(api_key,
                                       locationn,
                                       start_date,
                                       end_date,
                                       frequency,
                                       location_label = False,
                                       export_csv = False,
                                       store_df = True)
    tablo = pd.DataFrame(data = hist_weather_data[0] )
    path= "A.xlsx"
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    writer.book = book
    tablo.to_excel(writer,location)
    writer.save()
    #print(type(hist_weather_data))
    writer.close()
    return frequency
for location in lokasyonlar:
    hava_cek(location)



Retrieving weather data for usak


Currently retrieving data for usak: from 2018-01-01 to 2018-01-31
Time elapsed (hh:mm:ss.ms) 0:00:01.448900
Currently retrieving data for usak: from 2018-02-01 to 2018-02-28
Time elapsed (hh:mm:ss.ms) 0:00:02.881188
Currently retrieving data for usak: from 2018-03-01 to 2018-03-31
Time elapsed (hh:mm:ss.ms) 0:00:04.038832
Currently retrieving data for usak: from 2018-04-01 to 2018-04-30
Time elapsed (hh:mm:ss.ms) 0:00:06.721272
Currently retrieving data for usak: from 2018-05-01 to 2018-05-31
Time elapsed (hh:mm:ss.ms) 0:00:08.328976
Currently retrieving data for usak: from 2018-06-01 to 2018-06-30
Time elapsed (hh:mm:ss.ms) 0:00:10.448428
Currently retrieving data for usak: from 2018-07-01 to 2018-07-31
Time elapsed (hh:mm:ss.ms) 0:00:12.577189
Currently retrieving data for usak: from 2018-08-01 to 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:00:14.897839
Currently retrieving data for usak: from 2018-09-01 to 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:00:1